# Quantizing LeNet-5 model
This notebook shows how to quantize a pre-trained Fireball model using Codebook Quantization. It assumes 
that a trained LeNet-5 model already exists in the ```Models``` directory. You can use the notebook
[Handwritten Digit recognition (LeNet-5/MNIST)](LeNet5-MNIST.ipynb) to create and train a LeNet-5 model.

If you want to quantize a Low-Rank model, you can use [this](LeNet5-MNIST-Reduce.ipynb) notebook
to reduce the number of parameters in LeNet-5.

Model quantization reduces the size of the model by using less number of bits for each floating 
point parameter. Fireball uses a codebook quantization method based on K-Means clustering algorithm.

```quantizeModel``` is a class method that receives the file names of input and output to the 
quantization process. It also receives the quantization parameters such as ```minBits```, ```maxBits```, 
```mse```, and ```pdfFactor```.

Fireball can create models with 2-bit to 12-bit quantization (Codebook sizes 4 to 4096). For the quantized
model to be compatible with ```CoreML```, we need to make sure the codebook size is a power of 2, less than or equal to 256, and only "weight" parameters are quantized (not biases)


## Quantizing a pretrained model
The code in the following cell quantizes the model specified by ```orgFileName``` and creates a
new quantized model.

For each parameter tensor of the model, we try quantization bits 2 to 8 and find the best quantization
that satisfies the specified MSE value.

To get better quantization (smaller model) increase ```mse```; to get better performance (larger model)
use a smaller ```mse```.

In [1]:
from fireball import Model

# orgFileName = "Models/LeNet5.fbm"        # Original model
# orgFileName = "Models/LeNet5P.fbm"       # Pruned
# orgFileName = "Models/LeNet5PR.fbm"      # Pruned - Retrained
# orgFileName = "Models/LeNet5R.fbm"       # Reduced
# orgFileName = "Models/LeNet5RP.fbm"      # Reduced - Pruned
# orgFileName = "Models/LeNet5RP.fbm"      # Reduced - Pruned - Retrained
# orgFileName = "Models/LeNet5RR.fbm"      # Reduced - Retrained
# orgFileName = "Models/LeNet5RRP.fbm"     # Reduced - Retrained - Pruned
orgFileName = "Models/LeNet5RRPR.fbm"    # Reduced - Retrained - Pruned - Retrained

quantizedFileName = orgFileName.replace('.fbm', 'Q.fbm')  # Append 'Q' to the filename for "Quantized"

# quantizing the model
qResults = Model.quantizeModel(orgFileName, quantizedFileName, 
                               mseUb=.001, minBits=2, maxBits=8, reuseEmptyClusters=True, weightsOnly=True,
                               quiet=False, verbose=True, numWorkers=0)


Reading model parameters from "Models/LeNet5RRPR.fbm" ... Done.
Quantizing 13 tensors ... 
   Quantization Parameters:
        mseUb .............. 0.001
        pdfFactor .......... 0.1
        reuseEmptyClusters . True
        weightsOnly ........ True
        minBits ............ 2
        maxBits ............ 8
    Tensor 1 of 13 Shape: 5x5x1x6 ........... Quantized. (16 clusters - MSE: 0.0006)
    Tensor 2 of 13 Shape: 6 ................. Ignored. (1-D Tensor)
    Tensor 3 of 13 Shape: 5x5x6x8 ........... Quantized. (16 clusters - MSE: 0.0004)
    Tensor 4 of 13 Shape: 1x1x8x16 .......... Quantized. (32 clusters - MSE: 0.0003)
    Tensor 5 of 13 Shape: 16 ................ Ignored. (1-D Tensor)
    Tensor 6 of 13 Shape: 400x8 ............. Quantized. (16 clusters - MSE: 0.0010)
    Tensor 7 of 13 Shape: 8x120 ............. Quantized. (32 clusters - MSE: 0.0002)
    Tensor 8 of 13 Shape: 120 ............... Ignored. (1-D Tensor)
    Tensor 9 of 13 Shape: 120x8 ............. Quantiz

Compare the data and file sizes before and after quantization.
## Evaluate the quantized model

In [2]:
from fireball.datasets.mnist import MnistDSet

testDs = MnistDSet.makeDatasets('test', batchSize=128)

model = Model.makeFromFile(quantizedFileName, testDs=testDs, gpus='0')   
model.initSession()

results = model.evaluate()


Reading from "Models/LeNet5RRPRQ.fbm" ... Done.
Creating the fireball model "LeNet-5" ... Done.
Metal device set to: Apple M1 Max
  Processed 10000 Sample. (Time: 0.62 Sec.)                              

Observed Accuracy: 0.988700
Expected Accuracy: 0.100361
Kappa: 0.987439 (Excellent)


## Re-training after quantization
Fireball can retrain the quantized models by modifying (learning) the quantization codebooks. The following cell creates a "tune" dataset by sampling from the training dataset and uses it to "fine-tune" the quantized model for 5 epochs. The re-trained model is then evaluated and saved to the ```Models``` directory.

In [3]:
tuneDs,validDs = MnistDSet.makeDatasets('tune,valid', batchSize=128)
print(tuneDs)

model = Model.makeFromFile(quantizedFileName, 
                           trainDs=tuneDs, validationDs=validDs, # Use the "tuneDs" for training
                           numEpochs=5,
                           learningRate=(1e-3,1e-5),
                           optimizer="Momentum",
                           gpus='0')
model.initSession()
model.train()

model.evaluateDSet(testDs)

retrainedFileName = quantizedFileName.replace('.fbm', 'R.fbm')  # Append 'R' to the filename for "Re-trained"
model.save(retrainedFileName)   # Save the re-trained model to the "Models" directory

MnistDSet Dataset Info:
    Dataset Name ................................... tune
    Dataset Location ............................... /Users/shahab/data/mnist/
    Number of Classes .............................. 10
    Number of Samples .............................. 5400
    Sample Shape ................................... (28, 28, 1)


Reading from "Models/LeNet5RRPRQ.fbm" ... Done.
Creating the fireball model "LeNet-5" ... Done.
+--------+---------+---------------+-----------+-------------------+
| Epoch  | Batch   | Learning Rate | Loss      | Valid/Test Error  |
+--------+---------+---------------+-----------+-------------------+
| 1      | 42      | 0.00034056156 | 0.0478891 |    1.67% N/A      |
| 2      | 85      | 0.00011598216 | 0.0433467 |    1.33% N/A      |
| 3      | 128     | 0.00003752411 | 0.0424052 |    1.33% N/A      |
| 4      | 171     | 0.00001277927 | 0.0418267 |    1.33% N/A      |
| 5      | 214     | 0.00000413452 | 0.0417304 |    1.33% N/A      |
+--------+

## Where do I go from here?

[Exporting LeNet-5 Model to ONNX](LeNet5-MNIST-ONNX.ipynb)

[Exporting LeNet-5 Model to TensorFlow](LeNet5-MNIST-TF.ipynb)

[Exporting LeNet-5 Model to CoreML](LeNet5-MNIST-CoreML.ipynb)

[Hand-written Digit Recognition as a Regression problem](Regression.ipynb)

---

[Fireball Playgrounds](../Contents.ipynb)

[Handwritten Digit Recognition (LeNet-5/MNIST)](LeNet5-MNIST.ipynb)

[Reducing number of parameters of LeNet-5 Model](LeNet5-MNIST-Reduce.ipynb)

[Pruning LeNet-5 Model](LeNet5-MNIST-Prune.ipynb)
